In [1]:
import numpy as np
import pandas as pd
import os
import nltk

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/miglani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [3]:
block_words = ['newsgroups', 'xref', 'path', 'from', 'subject', 'sender', 'organisation', 'apr','gmt', 'last','better','never','every','even','two','good','used','first','need','going','must','really','might','well','without','made','give','look','try','far','less','seem','new','make','many','way','since','using','take','help','thanks','send','free','may','see','much','want','find','would','one','like','get','use','also','could','say','us','go','please','said','set','got','sure','come','lot','seems','able','anything','put', '--', '|>', '>>', '93', 'xref', 'cantaloupe.srv.cs.cmu.edu', '20', '16', "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'", '21', '19', '10', '17', '24', 'reply-to:', 'thu', 'nntp-posting-host:', 're:','25''18'"i'd"'>i''22''fri,''23''>the','references:','xref:','sender:','writes:','1993','organization:']

In [4]:
## Download the dataset
import urllib.request
urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/20_newsgroups.tar.gz", "a.tar.gz")

('a.tar.gz', <http.client.HTTPMessage at 0x7efd584ddda0>)

In [5]:
#Extracting the dataset
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()

In [6]:
##Make a list of the folders in the dataset
directory = [f for f in os.listdir('./20_newsgroups') if not f.startswith('.')]
directory

['talk.religion.misc',
 'sci.electronics',
 'comp.windows.x',
 'talk.politics.mideast',
 'soc.religion.christian',
 'comp.sys.mac.hardware',
 'rec.motorcycles',
 'comp.graphics',
 'sci.crypt',
 'rec.autos',
 'rec.sport.baseball',
 'talk.politics.misc',
 'sci.space',
 'misc.forsale',
 'rec.sport.hockey',
 'comp.os.ms-windows.misc',
 'talk.politics.guns',
 'comp.sys.ibm.pc.hardware',
 'sci.med',
 'alt.atheism']

In [7]:
# Create a dictionary of words with their frequency
vocab = {}
for i in range(len(directory)):
    #Create a list of files in the given dictionary 
    files = os.listdir('./20_newsgroups/' + directory[i])
 
    for j in range(len(files)):
        #Path of each file 
        path = './20_newsgroups/' + directory[i] + '/' + files[j]
        
        #open the file and read it
        text = open(path, 'r', errors='ignore').read()
        
        for word in text.split():
            if len(word) != 1: 
                #Check if word is a non stop word or non block word(we have created) only then proceed
                if not word.lower() in stop_words:
                    if not word.lower() in block_words:     
                        #If word is already in dictionary then we just increment its frequency by 1
                        if vocab.get(word.lower()) != None:
                            vocab[word.lower()] += 1

                        #If word is not in dictionary then we put that word in our dictinary by making its frequnecy 1
                        else:
                            vocab[word.lower()] = 1
            
# vocab

In [8]:
import operator
sorted_vocab = sorted(vocab.items(), key= operator.itemgetter(1), reverse= True)
# sorted_vocab

In [9]:
# Dictionary containing the most occuring k-words.
kvocab={}

# Frequency of 1000th most occured word
z = sorted_vocab[2000][1]

for x in sorted_vocab:
    kvocab[x[0]] = x[1]
    
    if x[1] <= z:
        break

In [10]:
sorted_vocab[0:100]

[('subject:', 20486),
 ('from:', 20417),
 ('date:', 20137),
 ('newsgroups:', 20081),
 ('message-id:', 20050),
 ('lines:', 20042),
 ('path:', 20029),
 ('article', 12108),
 ('people', 8415),
 ('university', 8203),
 ('know', 7695),
 ('think', 7205),
 ("i'm", 5823),
 ('distribution:', 4406),
 ('time', 4336),
 ('it.', 4185),
 ('anyone', 3976),
 ('world', 3602),
 ('right', 3326),
 ('believe', 3309),
 ('still', 3290),
 ('something', 3190),
 ('computer', 3157),
 ('system', 3137),
 ("i've", 3114),
 ('god', 2881),
 ('15', 2881),
 ('back', 2840),
 ("can't", 2836),
 ('news', 2836),
 ('state', 2787),
 ('work', 2692),
 ('>in', 2610),
 ('someone', 2610),
 ('government', 2534),
 ('problem', 2528),
 ('23', 2522),
 ('read', 2516),
 ('another', 2516),
 ('usa', 2496),
 ('information', 2480),
 ('>the', 2452),
 ('number', 2424),
 ("that's", 2382),
 ('things', 2378),
 ('part', 2323),
 ('fri,', 2307),
 ('point', 2297),
 ('little', 2294),
 ('22', 2284),
 ('windows', 2265),
 ('>i', 2253),
 ('tue,', 2241),
 ('fi

In [11]:
features_list = list(kvocab.keys())

# Create a Dataframe containing features_list as columns 
df = pd.DataFrame(columns = features_list)

# Filling the x_train values in dataframe 

for i in range(len(directory)):
    #Create a list of files in the given dictionary 
    files = os.listdir('./20_newsgroups/' + directory[i])
 
    for j in range(len(files)):
        #Insert a row at the end of Dataframe with all zeros
        df.loc[len(df)] = np.zeros(len(features_list))
        
        #Path of each file 
        path = './20_newsgroups/' + directory[i] + '/' + files[j]
        
        text = open(path, 'r', errors='ignore').read()
        
        
        for word in text.split():
            if word.lower() in features_list:
                df[word.lower()][len(df)-1] += 1
                

# df.head()

In [12]:
# Making the 2d array of x
x = df.values

# Feature list
f_list = list(df)

In [13]:
# Creating  y array containing labels for classification 

y = []

for i in range(len(directory)):
    #Create a list of files in the given dictionary 
    files = os.listdir('./20_newsgroups/' + directory[i])
 
    for j in range(len(files)):
        y.append(i)

y = np.array(y)
y.shape

(19997,)

In [14]:
# Splitting the whole dataset for training and testing
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.25, random_state = 0)

In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

train_score = clf.score(x_train, y_train)
test_score = clf.score(x_test, y_test)

train_score, test_score

(0.8772421150896846, 0.834)

In [16]:
#implementing from scratch
def fit(x_train, y_train):
    
    # dictionary containing the count of words
    count = {}
       
    #set of all classes 
    set_class = set(y_train)
            
    for current_class in set_class:
        count[current_class] = {}
        count["total_data"] = len(y_train)
        
        #Rows whose class is current_class
        current_class_rows = (y_train == current_class)
        
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        
        sums = 0
        for i in range(len(f_list)):
            # For each class, calculating total frequency of a feature 
            count[current_class][f_list[i]] = x_train_current[:,i].sum()
            sums = sums + count[current_class][f_list[i]]
        
        #Calculating total count of words of a class
        count[current_class]["total_count"] = sums
        
    return count

In [17]:
def probability(dictionary, row, current_class):
    # class_prob = log of probability of the current class = log(no of documents having class as current_class)/ (total number of documents)
    class_prob = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    total_prob = class_prob
    
    
    for i in range(len(row)):
        #Numerator
        word_count = dictionary[current_class][f_list[i]] + 1     
        # Denominator
        total_count = dictionary[current_class]["total_count"] + len(f_list)
        #Add 1 to numerator and len(row) in denominator for laplace correction
        
        #Log Probabilty of a word 
        word_prob = np.log(word_count) - np.log(total_count)
        
        #Calculating probability frequency number of times
        for j in range(int(row[i])):
            total_prob += word_prob
        
    return total_prob

In [18]:
def predictSinglePoint(row, dictionary):
    classes = dictionary.keys()
    
    #Initialising best_prob and best_class as very low count
    
    best_prob = -1000
    best_class = -1
    first_iter = True
    
    for current_class in classes:
        if(current_class == "total_data"):
            continue
        
        #Calculating probabilty that the given row belong to current_class
        prob_current_class = probability(dictionary, row, current_class)
        
        #For first iteration I set the best_prob to be the probabilty that row is of first class and best_class to be first class
        #For rest iteration, I checked if the probabilty that row is of the current_class is greater than the best_prob then we update best_prob and best_class.
        if(first_iter or prob_current_class > best_prob):
            best_prob = prob_current_class
            best_class = current_class
        
        first_iter = False
    
    # Return the best class which has maximum probabilty.
    return best_class

In [19]:
def predict(x_test, dictionary):
    # Initialise a list which contain the predictions
    y_pred_self = []
    
    #Iterate through each row in x_test
    for j in range(len(x_test)):
        
        #Calculate the prediction of the class to which the row belong to.
        pred_class = predictSinglePoint(x_test[j,:], dictionary) 
        
        #Append the predicted class to our list
        y_pred_self.append(pred_class)
    
    #Return the list of predictions
    return y_pred_self

In [20]:
#Training the model 
dictionary = fit(x_train, y_train)

#Testing the model 
y_pred_self = predict(x_test, dictionary)

In [21]:
from sklearn.metrics import accuracy_score
print("Accuracy for sklearn MultinomialNB() - ", test_score)
print("Accuracy for self-implemented Naive Bayes - ", accuracy_score(y_test, y_pred_self))

Accuracy for sklearn MultinomialNB() -  0.834
Accuracy for self-implemented Naive Bayes -  0.8366


In [22]:
#classification report of both the algorithms 
from sklearn.metrics import classification_report
print("Classification report for sklearn MultinomialNB()",classification_report(y_test, y_pred))
print("Classification report for self-implemented Naive Bayes ",classification_report(y_test, y_pred_self))

Classification report for sklearn MultinomialNB()              precision    recall  f1-score   support

          0       0.65      0.46      0.54       233
          1       0.75      0.88      0.81       253
          2       0.91      0.81      0.86       249
          3       0.94      0.82      0.88       240
          4       0.96      0.99      0.97       236
          5       0.82      0.89      0.85       239
          6       0.84      0.97      0.90       260
          7       0.84      0.77      0.81       271
          8       0.97      0.86      0.91       283
          9       0.81      0.88      0.85       247
         10       0.92      0.94      0.93       232
         11       0.72      0.53      0.61       234
         12       0.90      0.88      0.89       243
         13       0.75      0.83      0.79       257
         14       0.97      0.94      0.96       245
         15       0.76      0.87      0.81       253
         16       0.72      0.91      0.80      